### Importing required libraries

In [1]:
from torch.utils.data import DataLoader
from torchtext.datasets import IMDB
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
import torch 
import torch.nn as nn
from transformers import XLNetTokenizer

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torchtext\datasets\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORT

### Loading the dataset

In [2]:
train_iter,valid_iter=IMDB()

C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [3]:
next(iter(train_iter))
next(iter(train_iter))

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

### Loading XLNetTokenizer and building vocabulary

In [4]:
tokenizer=XLNetTokenizer.from_pretrained('xlnet-base-cased')
def yield_tokens(train_iter):
    for _,data_sample in train_iter:
            yield tokenizer.tokenize(data_sample)
vocab=build_vocab_from_iterator(yield_tokens(train_iter),specials=["<unk>","<pad>",'<|endoftext|>'],special_first=True)
vocab.set_default_index(vocab['<unk>'])

In [5]:
index_to_string=vocab.get_itos()

### Making of DataLoader

In [6]:
from torchtext.vocab import build_vocab_from_iterator
import torch

# 1. Final get_sample function
def get_sample(block_size, tokens_list):
    random_start = 0
    stop = block_size
    
    # Slicing tokens
    src_sequence = tokens_list[random_start:stop]
    tgt_sequence = tokens_list[random_start + 1:stop + 1]
    
    # List conversion to avoid attribute errors and maintain alignment
    src_sequence = list(src_sequence)
    tgt_sequence = list(tgt_sequence)
    
    # Padding if target is short
    if len(tgt_sequence) < len(src_sequence):
        tgt_sequence.append('<|endoftext|>')
        
    return src_sequence, tgt_sequence

# 2. Final implementation logic
# train_iter se ek sample nikalna
label, text = next(iter(train_iter))

# IMPORTANT: .tokenize() use karein, sirf tokenizer(text) nahi
tokens = tokenizer.tokenize(text)

# Sequence generate karein
block_size = 10
src_text, tgt_text = get_sample(block_size, tokens)

# 3. Numericalization (Ab error nahi aayega kyunki ye list of strings hai)
src_indices = vocab(src_text)
tgt_indices = vocab(tgt_text)

# 4. Convert to Tensors
src_tensor = torch.tensor(src_indices, dtype=torch.long)
tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long)

print(f"Source Indices: {src_indices}")
print(f"Target Indices: {tgt_indices}")

Source Indices: [19, 1525, 19, 9566, 658, 6872, 9990, 3714, 22, 453]
Target Indices: [1525, 19, 9566, 658, 6872, 9990, 3714, 22, 453, 4556]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    a

In [7]:
BATCH_SIZE=1

batch_of_tokens=[]

for i in range(BATCH_SIZE):
  _,text =next(iter(train_iter))
  batch_of_tokens.append(tokenizer(text))

In [8]:
text=batch_of_tokens[0][0:100]


In [9]:
# Initialize empty lists to store source and target sequences
src_batch, tgt_batch = [], []

# Define the batch size
BATCH_SIZE = 2

# Loop to create batches of source and target sequences
for i in range(BATCH_SIZE):
    # Retrieve the next data point from the training iterator
    _,text = next(iter(train_iter))

    # Generate source and target sequences using the get_sample function
    src_sequence_text, tgt_sequence_text = get_sample(block_size, tokenizer(text))

    # Convert source and target sequences to tokenized vocabulary indices
    src_sequence_indices = vocab(src_sequence_text)
    tgt_sequence_indices = vocab(tgt_sequence_text)

    # Convert the sequences to PyTorch tensors with dtype int64
    src_sequence = torch.tensor(src_sequence_indices, dtype=torch.int64)
    tgt_sequence = torch.tensor(tgt_sequence_indices, dtype=torch.int64)

    # Append the source and target sequences to their respective batches
    src_batch.append(src_sequence)
    tgt_batch.append(tgt_sequence)

    # Print the output for every 2nd sample (adjust as needed)
    print(f"Sample {i}:")
    print("Source Sequence (Text):", src_sequence_text)
    print("Source Sequence (Indices):", src_sequence_indices)
    print("Source Sequence (Shape):", src_sequence.shape)
    print("Target Sequence (Text):", tgt_sequence_text)
    print("Target Sequence (Indices):", tgt_sequence_indices)
    print("Target Sequence (Shape):", tgt_sequence.shape)

Sample 0:
Source Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Source Sequence (Indices): [0, 0, 0]
Source Sequence (Shape): torch.Size([3])
Target Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Target Sequence (Indices): [0, 0, 0]
Target Sequence (Shape): torch.Size([3])
Sample 1:
Source Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Source Sequence (Indices): [0, 0, 0]
Source Sequence (Shape): torch.Size([3])
Target Sequence (Text): ['input_ids', 'token_type_ids', 'attention_mask']
Target Sequence (Indices): [0, 0, 0]
Target Sequence (Shape): torch.Size([3])


In [34]:
def collate_batch(sample):
    label, text = sample
    tokens = tokenizer(text)
    src_sequence, _ = get_sample(BLOCK_SIZE, tokens)
    src_sequence = vocab(src_sequence)
    text = torch.tensor(src_sequence, dtype=torch.long).unsqueeze(0)
    label = torch.tensor(label, dtype=torch.long)
    return text, label


In [11]:
train_list = list(train_iter)
valid_list = list(valid_iter)

train_dataloader = DataLoader(
    train_list,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_batch
)

valid_dataloader = DataLoader(
    valid_list,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_batch
)


### Masking

In [12]:
def generate_mask(size):
    mask=torch.tril((torch.ones(size,size))==1)
    mask=mask.float().masked_fill(mask==0,float('-inf')).masked_fill(mask==1,float(0.0))
    return mask

In [13]:
def create_mask(src):
    seq_len=src.shape[0]
    src_mask=generate_mask(seq_len)
    src_padding_mask=(src==vocab("<pad>")).transpose(0,1)
    return src_mask,src_padding_mask

### Position Encoding

In [14]:
class Position_encoder(nn.Module):
    def __init__(self,vocab_size,d_model,dropout):
        super(Position_encoder,self).__init__()
        pe=torch.zeros(vocab_size,d_model)
        pos=torch.arange(0,vocab_size)
        pos=pos.view(-1,1)
        div_term=1/(10000**(2*torch.arange(0,d_model/2)/d_model))
        div_term=div_term.view(1,-1)
        pe[:,0::2]=torch.sin(pos*div_term)
        pe[:,1::2]=torch.cos(pos*div_term)
        self.pe=pe
        self.dropout=nn.Dropout(dropout)
    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pe[:token_embedding.size(0), :])

### Token Embedding

In [15]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

### Define GPT Like Model Architecture

In [44]:
class GPT_Like_model(nn.Module):
    def __init__(self, embed_size, vocab_size, num_heads, num_layers, pad_idx, dropout=0.1):
        super().__init__()

        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=pad_idx)
        self.positional_encoder = Position_encoder(d_model=embed_size,vocab_size=vocab_size,dropout=dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_size,
            nhead=num_heads,
            dropout=dropout
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.pad_idx = pad_idx

    def create_mask(self, src):
        seq_len, batch_size = src.size()
    
        src_mask = torch.triu(
            torch.ones(seq_len, seq_len, device=src.device) * float("-inf"),
            diagonal=1
        )
    
        src_padding_mask = torch.zeros(batch_size, seq_len, device=src.device).bool()
    
        return src_mask, src_padding_mask

    def forward(self, x):
        x = x.transpose(0, 1)
    
        seq_len, batch_size = x.size()
    
        src_mask = torch.triu(
            torch.ones(seq_len, seq_len, device=x.device) * float("-inf"),
            diagonal=1
        )
    
        x = self.embed(x)
        x = self.positional_encoder(x)
    
        x = self.encoder(x, mask=src_mask)
    
        x = x.mean(dim=0)
        x = self.fc_out(x)
        return x
    


In [45]:
ntokens = len(vocab)
emsize = 200
nlayers = 2
nhead = 2  
pad_idx=1
dropout = 0.2
model = GPT_Like_model(embed_size=emsize, num_heads=nhead, num_layers=nlayers, vocab_size=ntokens,pad_idx=pad_idx,dropout=dropout)

### Prompting

In [19]:
BLOCK_SIZE=30
def encode_prompt(prompt, block_size=BLOCK_SIZE):
    # Handle None prompt
    while prompt is None:
        prompt = input("Sorry, prompt cannot be empty. Please enter a valid prompt: ")

    tokens = tokenizer.tokenize(prompt)
    number_of_tokens = len(tokens)

    # Handle long prompts
    if number_of_tokens > block_size:
        tokens = tokens[-block_size:]  # Keep last block_size characters

    prompt_indices = vocab(tokens)
    prompt_encoded = torch.tensor(prompt_indices, dtype=torch.int64).reshape(-1, 1)
    return prompt_encoded

In [20]:
print([index_to_string[token] for token in encode_prompt(
    "This is a prompt to get model generate next words.")])


['▁This', '▁is', '▁a', '▁prompt', '▁to', '▁get', '▁model', '▁generate', '▁next', '▁words', '.']


In [21]:

#auto-regressive Language Model text generation
def generate(model, prompt=None, max_new_tokens=500, block_size=BLOCK_SIZE, vocab=vocab, tokenizer=tokenizer):

    # Encode the input prompt using the provided encode_prompt function
    prompt_encoded = encode_prompt(prompt)
    tokens = []

    # Generate new tokens up to max_new_tokens
    for _ in range(max_new_tokens):
        # Decode the encoded prompt using the model's decoder
        logits = model(prompt_encoded)

        # Transpose the logits to bring the sequence length to the first dimension
        logits = logits.transpose(0, 1)

        # Select the logits of the last token in the sequence
        logit_prediction = logits[:, -1]

        # Choose the most probable next token from the logits(greedy decoding)
        next_token_encoded = torch.argmax(logit_prediction, dim=-1).reshape(-1, 1)


        # Append the next token to the prompt_encoded and keep only the last 'block_size' tokens
        prompt_encoded = torch.cat((prompt_encoded, next_token_encoded), dim=0)[-block_size:]

        # Convert the next token index to a token string using the vocabulary
        # Move the tensor back to CPU for vocab lookup if needed
        token_id = next_token_encoded.to('cpu').item()
        tokens.append(vocab.get_itos()[token_id])

    # Join the generated tokens into a single string and return
    return ' '.join(tokens)

In [22]:
generate(model,prompt="this is the beginning of",max_new_tokens=30,vocab=vocab,tokenizer=tokenizer)

'▁help ▁linebacker anov ▁presents ▁Faso ▁drunk ▁reverence ▁House lying ▁shook Land ▁Goddard jer ▁Neg ▁Avoid ▁goodness ▁mixture lick ▁lot ▁Resolution ▁maintain achi ewski ▁Nice pilot ▁Examples ▁Sen trans ▁deemed ito'

### Training 

In [23]:
loss_fn = nn.CrossEntropyLoss(ignore_index=1)

In [46]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.01, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10000, gamma=0.9)

def train(model: nn.Module,train_data) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 10000
    start_time = time.time()

    num_batches = len(list(train_data)) // block_size
    for batch,srctgt in enumerate(train_data):
        src= srctgt[0]
        tgt= srctgt[1]
        logits = model(src)
        logits_flat = logits.reshape(-1, logits.shape[-1])
        loss = loss_fn(logits_flat, tgt.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()

        if (batch % log_interval == 0 and batch > 0) or batch==42060:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            #cur_loss = total_loss / log_interval
            cur_loss = total_loss / batch
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch//block_size:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.4f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            start_time = time.time()

    return total_loss

In [ ]:
from tqdm import tqdm
import time
import math
import torch
from torchtext.datasets import IMDB

best_val_loss = float("inf")
epochs = 30
Train_losses = []
Val_losses = []

for epoch in tqdm(range(1, epochs + 1), desc="Epochs", position=0):
    epoch_start_time = time.time()

    train_iter, valid_iter = IMDB(split=("train", "test"))

    model.train()
    train_loss = 0.0
    train_steps = 0

    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", leave=False, position=1):
        text, labels = collate_batch(batch)
        text = text
        labels = labels
        optimizer.zero_grad()
        output = model(text)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_steps += 1

    train_loss /= train_steps

    model.eval()
    val_loss = 0.0
    val_steps = 0

    with torch.no_grad():
        for batch in tqdm(valid_iter, desc=f"Validation Epoch {epoch}", leave=False, position=2):
            text, offsets, labels = collate_batch(batch)
            text = text
            offsets = offsets
            labels = labels

            output = model(text, offsets)
            loss = loss_fn(output, labels)

            val_loss += loss.item()
            val_steps += 1

    val_loss /= val_steps
    val_ppl = math.exp(val_loss)

    Train_losses.append(train_loss)
    Val_losses.append(val_loss)

    elapsed = time.time() - epoch_start_time

    tqdm.write("-" * 89)
    tqdm.write(
        f"end of epoch {epoch:3d} | time: {elapsed:5.2f}s | "
        f"train loss {train_loss:5.2f} | valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}"
    )
    tqdm.write("-" * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "model_best_val_loss.pt")
